# Exploratory Data Analysis general


### Loading and Exploring the data and checking for cleaning

In [8]:
import regex as re
import pandas as pd

In [9]:
## Data Load 


# Load the data from the CSV file
file_path = r"C:\Users\MicrobeJ\Downloads\data.csv"
data = pd.read_csv(file_path)

# Check the structure of the dataset (rows and columns)
data_shape = data.shape

# Identify the different data types (classes) present in each column
data_types = data.dtypes

# Calculate the number of unique values in each column, excluding the header
unique_values_per_column = data.nunique()

# Organizing the information into a dataframe for better readability
data_structure_info = pd.DataFrame({
    'Data Types': data_types,
    'Unique Values': unique_values_per_column
})

print(data_shape, data_structure_info)

# Checking for columns with 0 unique values to see if they contain only NaNs or a constant value
empty_or_constant_columns = data.loc[:, data.nunique() == 0]

# Checking for any missing values across the dataset
missing_values = data.isnull().sum()

# Checking for any values that are exactly 0 across the dataset
zero_values = (data == 0).sum()

# Combine the information into a dataframe for better readability
missing_zero_info = pd.DataFrame({
    'Missing Values': missing_values,
    'Zero Values': zero_values
})

print(empty_or_constant_columns, missing_zero_info)




(485, 59)                        Data Types  Unique Values
NAME                       object            485
NAME.id                    object            485
NAME.name                  object            485
ASSOCIATION                 int64              1
CLUMP                       int64              1
EXPERIMENT                 object              1
EXPERIMENT.count            int64              1
EXPERIMENT.date            object              1
EXPERIMENT.description    float64              0
EXPERIMENT.field            int64              1
EXPERIMENT.fullname        object              1
EXPERIMENT.id             float64              0
EXPERIMENT.index          float64              0
EXPERIMENT.name            object              1
EXPERIMENT.series           int64              1
IMAGE                      object              1
IMAGE.label                object              5
IMAGE.meta                 object              5
IMAGE.name                 object              1
INTENSITY 

The IMAGE.meta column entries appear to encode the Z stack information with a pattern c:x/y z:x/y t:x/y, where c represents the channel, z the Z stack level, and t the time point within the metadata string. We will focus on extracting the Z stack level (z:x/y) for our function. 

### Subset of the Data for Analysis

In [ ]:
# Making of the subset of the data only pulling possible useful varaibles

# Creating a subset of the data with the specified columns
subset_columns = [
    'NAME.id', 'NAME.name', 'EXPERIMENT.count', 'IMAGE.meta', 'IMAGE.name', 
    'INTENSITY.ch1', 'INTENSITY.ch2', 'INTENSITY.ch3', 'LOCATION', 'LOCATION.center', 
    'LOCATION.dist', 'LOCATION.half', 'LOCATION.pole', 'LOCATION.side', 'LOCATION.x', 
    'LOCATION.y', 'MAXIMA', 'MAXIMA.Maxima1', 'MAXIMA.count', 'MEDIAL', 'POSITION', 
    'POSITION.channel', 'POSITION.frame', 'POSITION.position', 'POSITION.slice', 
    'PROFILE_MEDIAL', 'SHAPE', 'SHAPE.angularity', 'SHAPE.area', 'SHAPE.aspectRatio', 
    'SHAPE.circularity', 'SHAPE.curvature', 'SHAPE.feret', 'SHAPE.length', 
    'SHAPE.morphology', 'SHAPE.orientation', 'SHAPE.perimeter', 'SHAPE.pole', 
    'SHAPE.roundness', 'SHAPE.sinuosity', 'SHAPE.solidity', 'SHAPE.width', 'ZSCORE'
]


# Selecting the columns from the dataset
data_subset = data[subset_columns]

# Display the first few rows of the subset to confirm
data_subset.head()

# A general Remapping/Renaming for Data Extraction
def extract_z_stack(meta_string):
    """Extract the Z stack level from the IMAGE.meta string."""
    match = re.search(r'z:(\d+)/(\d+)', meta_string)
    return match.group(1) if match else None

def rename_z_stack(data, meta_col, z_stack_col, new_name_mapping):
    """Rename the Z stack levels based on metadata information.

    Args:
        data (DataFrame): The pandas DataFrame containing the data.
        meta_col (str): The column name of the metadata.
        z_stack_col (str): The column name of the Z stack levels.
        new_name_mapping (dict): A dictionary mapping the original Z stack levels to new names.

    Returns:
        DataFrame: The pandas DataFrame with the Z stack levels renamed.
    """
    # Extract the Z stack levels
    data[z_stack_col] = data[meta_col].apply(extract_z_stack)
    
    # Map the Z stack levels to new names using the provided mapping
    data[z_stack_col] = data[z_stack_col].map(new_name_mapping)
    
    return data



# Define a function to parse the 'IMAGE.name' column and extract the specified components
def parse_image_name(image_name):
    # Parse the date (first 8 characters are the date in YYYYMMDD format)
    date = image_name[:8]
    
    # Use a regular expression to find the strain identifier, which seems to be in the format LZ followed by numbers
    strain_match = re.search(r'(LZ\d+)', image_name)
    strain = strain_match.group(1) if strain_match else None
    
    # Use a regular expression to find the time point (number followed by 'min')
    time_match = re.search(r'(\d+)min', image_name)
    time = int(time_match.group(1)) if time_match else None
    
    # Use a regular expression to find the condition ('inf' or similar pattern)
    cond_match = re.search(r'min_([a-zA-Z]+)', image_name)
    cond = cond_match.group(1) if cond_match else None
    
    # Use a regular expression to find the frame number (after 'T=')
    frame_match = re.search(r'T=(\d+)', image_name)
    frame = int(frame_match.group(1)) if frame_match else None
    
    return date, strain, time, cond, frame

# Since we don't have specific names to map, let's create an example mapping
# This mapping will be placeholder until we know the actual naming scheme you would like
example_new_name_mapping = {
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5
}


# Define a function to extract intensity statistics from the 'INTENSITY.ch#' columns
def microbej_extract_int_statistics(data, channel):
    """
    Extracts intensity statistics from a given intensity channel and adds them to the dataframe.
    
    Parameters:
    data (DataFrame): The pandas DataFrame containing the intensity information.
    channel (str): The channel number as a string to extract statistics for.
    
    Returns:
    DataFrame: The original DataFrame with additional columns for intensity statistics.
    """
    # Convert the intensity channel column to string to ensure string operations can be performed
    intensity_str = data[f'INTENSITY.ch{channel}'].astype(str)
    
    # Extract statistics using regular expressions
    data[f'ch{channel}.mean'] = intensity_str.str.extract(r'mean=(\d+\.\d+)').astype(float)
    data[f'ch{channel}.min'] = intensity_str.str.extract(r'min=(\d+\.\d+)').astype(float)
    data[f'ch{channel}.max'] = intensity_str.str.extract(r'max=(\d+\.\d+)').astype(float)
    data[f'ch{channel}.stdv'] = intensity_str.str.extract(r'stdv=(\d+\.\d+)').astype(float)
    
    # Return the dataframe with the new columns
    return data

    
# Re-applying the corrections with .loc to ensure proper assignment

# Extract data using the parse_image_name function and assign the result to new columns
parsed_image_data = data_subset['IMAGE.name'].apply(parse_image_name)
data_subset[['date', 'strain', 'time', 'cond', 'frame']] = pd.DataFrame(parsed_image_data.tolist(), index=data_subset.index)


# Apply the intensity statistics extraction function
for channel in range(1, 4):  # Assuming there are 3 channels as per your data
    data_subset = microbej_extract_int_statistics(data_subset, str(channel))

# Re-apply the Z-stack extraction function with .loc to avoid any SettingWithCopyWarning
data_subset.loc[:, 'z_stack'] = data_subset['IMAGE.meta'].apply(extract_z_stack)

# Checking the resulting DataFrame structure and column names to ensure correctness
data_structure = data_subset.dtypes
data_colnames = data_subset.columns.tolist()

print(data_structure, data_colnames)

# Now you have a processed dataframe
df_processed = data_subset.copy()

print(df_processed.head())

## Violin Plot the Intensities

In [ ]:
#### Violin Plots Labeled by condition and time groupings 

# Plotting the violin plots with individual data points colored by 'time' and marker style based on 'condition'
plt.figure(figsize=(14, 6))

# Plot for mean
plt.subplot(1, 2, 1)
sns.violinplot(x='z_stack', y='mean', data=df_background_only, inner=None, color='lightgray')
sns.stripplot(x='z_stack', y='mean', data=df_background_only, hue='time', dodge=True,
              marker='o', alpha=0.5, edgecolor='gray',
              palette=sns.color_palette("hsv", len(df_background_only['time'].unique())))
# Modify markers based on condition
for condition in df_background_only['condition'].unique():
    subset = df_background_only[df_background_only['condition'] == condition]
    marker = 'x' if condition == 'inf' else 'o'
    sns.stripplot(x='z_stack', y='mean', data=subset, hue='time', dodge=True, 
                  marker=marker, alpha=0.5, edgecolor='gray',
                  palette=sns.color_palette("hsv", len(subset['time'].unique())))

plt.title('Mean Pixel Intensity by Z-stack')
plt.legend(title='Time', bbox_to_anchor=(1.05, 1), loc='upper left')

# Plot for standard deviation
plt.subplot(1, 2, 2)
sns.violinplot(x='z_stack', y='std_dev', data=df_background_only, inner=None, color='lightgray')
sns.stripplot(x='z_stack', y='std_dev', data=df_background_only, hue='time', dodge=True,
              marker='o', alpha=0.5, edgecolor='gray',
              palette=sns.color_palette("hsv", len(df_background_only['time'].unique())))
# Modify markers based on condition
for condition in df_background_only['condition'].unique():
    subset = df_background_only[df_background_only['condition'] == condition]
    marker = 'x' if condition == 'inf' else 'o'
    sns.stripplot(x='z_stack', y='std_dev', data=subset, hue='time', dodge=True, 
                  marker=marker, alpha=0.5, edgecolor='gray',
                  palette=sns.color_palette("hsv", len(subset['time'].unique())))

plt.title('Standard Deviation of Pixel Intensity by Z-stack')
plt.legend(title='Time', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()



In [11]:
# Adjusting the function to rename columns from 'int#' to 'ch#'
def microbej_extract_int_statistics(data, channel):
    """
    Extracts intensity statistics from a given intensity channel and adds them to the dataframe.
    
    Parameters:
    data (DataFrame): The pandas DataFrame containing the intensity information.
    channel (str): The channel number as a string to extract statistics for.
    
    Returns:
    DataFrame: The original DataFrame with additional columns for intensity statistics.
    """
    # Extract statistics
    data[f'ch{channel}.mean'] = data[f'INTENSITY.ch{channel}'].str.extract(r'mean=(\d+\.\d+)').astype(float)
    data[f'ch{channel}.min'] = data[f'INTENSITY.ch{channel}'].str.extract(r'min=(\d+\.\d+)').astype(float)
    data[f'ch{channel}.max'] = data[f'INTENSITY.ch{channel}'].str.extract(r'max=(\d+\.\d+)').astype(float)
    data[f'ch{channel}.stdv'] = data[f'INTENSITY.ch{channel}'].str.extract(r'stdv=(\d+\.\d+)').astype(float)
    
    # Return the dataframe with the new columns
    return data

# Apply the adjusted function to each channel for the entire dataset
for i in range(1, 4):  # Assuming there are 3 channels
    data_subset = microbej_extract_int_statistics(data_subset, str(i))

# Display the first few rows to confirm the new columns
data_subset[['ch1.mean', 'ch1.min', 'ch1.max', 'ch1.stdv', 'ch2.mean', 'ch2.min', 'ch2.max', 'ch2.stdv', 'ch3.mean', 'ch3.min', 'ch3.max', 'ch3.stdv']].head()


C:\Users\MicrobeJ\AppData\Local\Temp\ipykernel_3048\2965239145.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'ch{channel}.mean'] = data[f'INTENSITY.ch{channel}'].str.extract(r'mean=(\d+\.\d+)').astype(float)
C:\Users\MicrobeJ\AppData\Local\Temp\ipykernel_3048\2965239145.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'ch{channel}.min'] = data[f'INTENSITY.ch{channel}'].str.extract(r'min=(\d+\.\d+)').astype(float)
C:\Users\MicrobeJ\AppData\Local\Temp\ipykernel_3048\2965239145.py:16: Se

,ch1.mean,ch1.min,ch1.max,ch1.stdv,ch2.mean,ch2.min,ch2.max,ch2.stdv,ch3.mean,ch3.min,ch3.max,ch3.stdv
0,14197.771,9515.0,19888.0,3105.1120,18853.219,8898.0,27483.0,4243.8570,4635.9297,2458.0,6031.0,728.49835
1,13454.421,10518.0,20259.0,2176.9985,9703.965,7326.0,11510.0,998.7617,8178.2104,4550.0,10558.0,1556.45030
2,15142.330,10027.0,21852.0,3474.3198,11617.922,6859.0,30524.0,3804.9343,7309.2000,3403.0,10953.0,2144.85180
3,11937.072,9062.0,17243.0,2310.1047,8345.310,6825.0,9827.0,717.0390,5861.5454,2871.0,7928.0,1115.72470
4,11153.036,9184.0,15266.0,1471.9005,9310.500,7451.0,11066.0,817.6870,6719.4520,4207.0,9343.0,1357.06820


In [12]:
# 2 Condition Violin Plot Violin Plot, strplot function

import matplotlib.pyplot as plt
import seaborn as sns


def violin_strplot_twoconditionx_hue(df, metric, xcondition1, xcondition2, huecondition): #new_df_name='new_df'
    # Create a deep copy of the DataFrame
    df_copy = df.copy()
    
    # Create a new column combining 'xcondition1' and 'xcondition2'
    df_copy['xcondition1_xcondition2'] = df_copy[xcondition1].astype(str) + "_" + df_copy[xcondition2].astype(str)
    
    plt.figure(figsize=(15, 6))
    
    # Create the violin plot
    sns.violinplot(x='xcondition1_xcondition2', y=metric, data=df_copy, inner=None, dodge=True, color='gray', alpha=0.5)
    
    # Create the strip plot
    sns.stripplot(x='xcondition1_xcondition2', y=metric, data=df_copy, hue=huecondition, dodge=True, jitter=True, marker='o', alpha=0.5)
    
    plt.title(f'{metric.capitalize()} Intensity Across {xcondition1} and {xcondition2}, Colored by {huecondition}')
    plt.legend(title=huecondition, bbox_to_anchor=(1, 1), loc=2)
    plt.tight_layout()
    plt.show()
    
    #Save the new DataFrame to the global environment
    #globals()[new_df_name] = df_copy



ModuleNotFoundError: No module named 'seaborn'